## test bleach correction

In [ ]:
from IPython.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))
from pathlib import Path
from numpy import where
from pystripe.core import imread_tif_raw_png, glob_re, log1p_jit, expm1_jit, filter_subband, get_bleach_correction_filter
import matplotlib.pyplot as plt
from skimage.filters import threshold_otsu


files_path = Path(r"Y:\SmartSPIM_Data\2023_04_02\20230402_10_49_31_SM220712_01_LS_15x_800z\Ex_488_Em_525_MIP")
for file in glob_re(r"tiff?", files_path):
    img = imread_tif_raw_png(file)
    if img.min() < img.max():
        img_filtered = log1p_jit(img)
        threshold = threshold_otsu(img_filtered)
        print(threshold)
        if threshold < 3:
            continue
        
        img_filter = get_bleach_correction_filter(img_filtered, 0.000125, max_method=False)
        img_filtered = where(img_filtered > 3, img_filtered / img_filter, img_filtered) 
        
        img_destriped = expm1_jit(filter_subband(img_filtered, 200, 0, "db10", False))
        img_filtered = expm1_jit(img_filtered)
        
        fig, axes = plt.subplots(nrows=2, ncols=2, figsize=(10, 10), dpi=300)
        axes[0, 0].imshow(img, cmap='gray', vmin=0, vmax=511)
        axes[0, 0].set_title("original")
        axes[0, 1].imshow(img_filtered, cmap='gray', vmin=0, vmax=511)
        axes[0, 1].set_title("bleach corrected")
        axes[1, 0].imshow(img_destriped, cmap='gray', vmin=0, vmax=511)
        axes[1, 0].set_title("bleach corrected -> destriped")
        axes[1, 1].imshow(img_filter, cmap='gray', vmin=0, vmax=1)
        axes[1, 1].set_title("bleach filter")
        plt.tight_layout()
        plt.show()